In [1]:
import numpy as np
import pandas as pd

In [15]:
data = pd.read_csv('Chicago_Building_Energy_Benchmarking_raw.csv')
data.head()

,Data Year,ID,Property Name,Address,ZIP Code,Community Area,Primary Property Type,Gross Floor Area - Buildings (sq ft),Year Built,# of Buildings,...,All Other Fuel Use (kBtu),Site EUI (kBtu/sq ft),Source EUI (kBtu/sq ft),Weather Normalized Site EUI (kBtu/sq ft),Weather Normalized Source EUI (kBtu/sq ft),Total GHG Emissions (Metric Tons CO2e),GHG Intensity (kg CO2e/sq ft),Latitude,Longitude,Location
0,2014,101794,225 West Wacker,225 West Wacker Drive,60606,LOOP,Office,"721,992",1989.0,1,...,NaN,51,161,50,157,"7,449",10.32,41.886286,-87.635081,"(41.88628574, -87.63508062)"
1,2014,100447,211 E. Chicago Ave.,211 E. Chicago Ave.,60611,NEAR NORTH SIDE,Office,"394,230",1965.0,1,...,NaN,85,171,81,167,"4,047",10.27,41.896450,-87.622566,"(41.89644968, -87.62256575)"
2,2014,103641,33 North LaSalle Street,33 North LaSalle Street,60602,LOOP,Office,"477,171",1929.0,1,...,NaN,90,161,NaN,NaN,"4,524",9.48,41.882865,-87.632053,"(41.88286478, -87.63205333)"
3,2014,157011,LittleVilMpx-CPS,3120 S Kostner Ave,60623,SOUTH LAWNDALE,K-12 School,"290,134",2005.0,1,...,NaN,96,216,93,211,"3,849",13.26,41.835167,-87.735566,"(41.83516722, -87.73556631)"
4,2014,102544,Olive-Harvey College,10001 South Woodlawn Ave,60628,PULLMAN,College/University,"397,700",1981.0,1,...,NaN,78,159,75,156,"3,817",9.60,41.711115,-87.591219,"(41.71111526, -87.59121912)"


In [16]:
def remove_columns_Chicago(df):
    df.drop(['Property Name',
             'Address',
             'Community Area',
             '# of Buildings',
             'ENERGY STAR Score',
             'Total GHG Emissions (Metric Tons CO2e)',
             'GHG Intensity (kg CO2e/sq ft)',
             'Latitude',
             'Longitude',
             'Location'
           ], inplace=True, axis=1)     
    
def create_buildingtypes(df):
    df['BuildingType'] = None
    
    residential = ['Multifamily Housing', 'Residential Care Facility', 'Senior Care Community',]
    nonresidential = ['Office', 'Financial Office', 'Hospital (General Medical & Surgical)', 
                      'Enclosed Mall', 'Retail Store', 'Ambulatory Surgical Center', 'Fitness Center/Health Club/Gym',
                      'Automobile Dealership', 'Strip Mall', 'Medical Office', 'Supermarket/Grocery Store', 
                      'Repair Services (Vehicle, Shoe, Locksmith, etc.)', 'Hotel', 'Wholesale Club/Supercenter',
                     'Bank Branch', 'Other - Mall', 'Performing Arts', 'Movie Theater', 'Distribution Center',
                      'Outpatient Rehabilitation/Physical Therapy', 'Urgent Care/Clinic/Other Outpatient', 'Other - Lodging/Residential',
                      'Other - Services', 'Stadium (Open)', 'Lifestyle Center','Mixed Use Property']
    specialty = ['K-12 School', 'College/University', 'Prison/Incarceration', 'Courthouse',
                 'Other - Entertainment/Public Assembly', 'Library', 'Other - Specialty Hospital',
                 'Laboratory', 'Adult Education',  'Museum', 'Convention Center', 'Worship Facility', 
                 'Social/Meeting Hall', 'Residence Hall/Dormitory', 'Other - Public Services',
                 'Other - Education', 'Pre-school/Daycare', 'Indoor Arena', 'Other - Recreation', 'Other',]
    for i in range(len(df)):
        if df['Primary Property Type'][i] in residential:
            df.at[i, 'BuildingType'] = 'Residential'
        elif df['Primary Property Type'][i] in nonresidential:
            df.at[i, 'BuildingType'] = 'NonResidential'
        elif df['Primary Property Type'][i] in specialty:
            df.at[i, 'BuildingType'] = 'Specialty'
        else:
            df.at[i, 'BuildingType'] = 'Not Available'
            
def restructure_columns(df):
    df['City'] = 'Chicago'
    df['State'] = 'IL'

    df.rename({'ID':'BuildingID',
               'ZIP Code':'ZipCode',
               'Data Year':'DataYear',
               'Primary Property Type': 'PrimaryPropertyUse', 
               'Gross Floor Area - Buildings (sq ft)':'PropertyGFATotal',
               'Year Built':'YearBuilt',
               'Electricity Use (kBtu)':'Electricity(kBtu)',
               'Natural Gas Use (kBtu)':'NaturalGas(kBtu)',
               'District Steam Use (kBtu)':'SteamUse(kBtu)',
               'District Chilled Water Use (kBtu)':'ChilledWaterUse(kBtu)',
               'All Other Fuel Use (kBtu)':'OtherFuelUse(kBtu)',
               'Site EUI (kBtu/sq ft)':'SiteEUI(kBtu/sf)',
               'Source EUI (kBtu/sq ft)': 'SourceEUI(kBtu/sf)', 
               'Weather Normalized Site EUI (kBtu/sq ft)':'SiteEUIWN(kBtu/sf)',
               'Weather Normalized Source EUI (kBtu/sq ft)':'SourceEUIWN(kBtu/sf)',  
              }, axis='columns', inplace=True)


    df = df[['BuildingID',
             'DataYear',
             'City', 
             'State',
             'ZipCode',
             'BuildingType',
             'PrimaryPropertyUse',
             'YearBuilt',
             'PropertyGFATotal',
             'SiteEUI(kBtu/sf)',
             'SiteEUIWN(kBtu/sf)',
             'SourceEUI(kBtu/sf)',
             'SourceEUIWN(kBtu/sf)',
             'SteamUse(kBtu)',
             'Electricity(kBtu)',
             'NaturalGas(kBtu)',
             'ChilledWaterUse(kBtu)',
             'OtherFuelUse(kBtu)']]
    
    return df

def reduce_zipcodes(df):
    for i in range(len(df)):
        if len(df['ZipCode'][i]) > 5 :
            zipcode = df['ZipCode'][i]
            df.at[i, 'ZipCode'] = zipcode[0:5]
        else :
            continue
            
def remove_commas(df):
    column_names_1 = np.array(df.columns[7:])
    
    for i in range(len(column_names_1)):
        column = column_names_1[i]
        df[column] = pd.to_numeric(df[column].astype(str).str.replace(',',''), errors='coerce')
        
def convert_to_int(df):
    column_names_2 = ['DataYear', 'ZipCode', 'YearBuilt']
    
    for i in range(len(column_names_2)):
        column = column_names_2[i]
        df[column] = df[column].astype('int64')
        
        
def clean_data(df):
    remove_columns_Chicago(df)
    create_buildingtypes(df)
    df = restructure_columns(df)
    reduce_zipcodes(df)
    df.fillna(0, inplace=True)
    remove_commas(df)
    convert_to_int(df)
    
    return df

In [17]:
data = clean_data(data)
data.head()

,BuildingID,DataYear,City,State,ZipCode,BuildingType,PrimaryPropertyUse,YearBuilt,PropertyGFATotal,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),ChilledWaterUse(kBtu),OtherFuelUse(kBtu)
0,101794,2014,Chicago,IL,60606,NonResidential,Office,1989,721992.0,51.0,50.0,161.0,157.0,0.0,37073768.0,0.0,0.0,0.0
1,100447,2014,Chicago,IL,60611,NonResidential,Office,1965,394230.0,85.0,81.0,171.0,167.0,0.0,15403962.0,17932947.0,0.0,0.0
2,103641,2014,Chicago,IL,60602,NonResidential,Office,1929,477171.0,90.0,0.0,161.0,0.0,0.0,15112533.0,26999828.0,1025160.0,0.0
3,157011,2014,Chicago,IL,60623,Specialty,K-12 School,2005,290134.0,96.0,93.0,216.0,211.0,0.0,16030378.0,11819639.0,0.0,0.0
4,102544,2014,Chicago,IL,60628,Specialty,College/University,1981,397700.0,78.0,75.0,159.0,156.0,0.0,14710299.0,16218361.0,0.0,0.0


In [19]:
data.to_csv(r'C:\Users\cjros\DIRECT\Capstone\Data\EnergyConsumption\Chicago\Chicago_Energy_Consumption_cleaned.csv', index_label='UID')

In [20]:
data_consistant = data.copy()
data_consistant = data_consistant[data_consistant.groupby(['BuildingID'])['BuildingID'].transform('size') > 2]

In [23]:
data.to_csv(r'C:\Users\cjros\DIRECT\Capstone\Data\EnergyConsumption\Chicago\Chicago_Energy_Consumption_trimmed.csv', index_label='UID')